### ***Importante librairies***

In [7]:
!python --version

Python 3.7.15


In [8]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [9]:
#print("TF version:", tf.__version__)
print("Numpy" + np.__version__)
#print("Keras" + tf.keras.__version__)
print("Pandas" + pd.__version__)

Numpy1.21.6
Pandas1.3.5


In [10]:
df =pd.read_csv('/content/drive/MyDrive/My research data /raw_data_sensors_.csv')
df.head()

,Ti,Hi,To,Ho,m1,l,co2
0,24.2,69.6,22.1,67.2,24.0,994.0,804.0
1,24.1,70.1,22.0,67.4,29.0,965.0,790.0
2,23.9,70.4,21.8,63.0,33.0,1001.0,790.0
3,23.8,71.0,21.5,65.9,34.0,994.0,790.0
4,23.6,71.4,21.4,65.9,35.0,1002.0,786.0


In [11]:
def df_to_X_y(df, window_size=1):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [r for r in df_as_np[i:i+window_size]]
    X.append(row)
    label = [df_as_np[i+window_size][0], df_as_np[i+window_size][1]]
    y.append(label)
  return np.array(X), np.array(y)

X, y = df_to_X_y(df)
X.shape, y.shape

X_train, y_train = X[:15215], y[:15215]  
X_val, y_val = X[15215:19562], y[15215:19562]
X_test, y_test = X[19562:], y[19562:]
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((15215, 1, 7), (15215, 2), (4347, 1, 7), (4347, 2), (2172, 1, 7), (2172, 2))

In [12]:
X_train, y_train = X[:15215], y[:15215]  
X_val, y_val = X[15215:19562], y[15215:19562]
X_test, y_test = X[19562:], y[19562:]
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((15215, 1, 7), (15215, 2), (4347, 1, 7), (4347, 2), (2172, 1, 7), (2172, 2))

In [13]:
Ti_training_mean = np.mean(X_train[:, :, 0])
Ti_training_std = np.std(X_train[:, :, 0])

Hi_training_mean = np.mean(X_train[:, :, 1])
Hi_training_std = np.std(X_train[:, :, 1])

To_training_mean = np.mean(X_train[:, :, 2])
To_training_std = np.std(X_train[:, :, 2])

Ho_training_mean = np.mean(X_train[:, :, 3])
Ho_training_std = np.std(X_train[:, :, 3])

m1_training_mean = np.mean(X_train[:, :, 4])
m1_training_std = np.std(X_train[:, :, 4])

l_training_mean = np.mean(X_train[:, :, 5])
l_training_std = np.std(X_train[:, :, 5])

carbone_training_mean = np.mean(X_train[:, :, 6])
carbone_training_std = np.std(X_train[:, :, 6])

def preprocess_feature(X):
  X[:, :, 0] = (X[:, :, 0] - Ti_training_mean) / Ti_training_std
  X[:, :, 1] = (X[:, :, 1] - Hi_training_mean) / Hi_training_std
  X[:, :, 2] = (X[:, :, 2] - To_training_mean) / To_training_std
  X[:, :, 3] = (X[:, :, 3] - Ho_training_mean) / Ho_training_std
  X[:, :, 4] = (X[:, :, 4] - m1_training_mean) / m1_training_std
  X[:, :, 5] = (X[:, :, 5] - l_training_mean) / l_training_std
  X[:, :, 6] = (X[:, :, 6] - carbone_training_mean) / carbone_training_std
  return X

def preprocess_output(y):
  y[:, 0] = (y[:, 0] - Ti_training_mean) / Ti_training_std
  y[:, 1] = (y[:, 1] - Hi_training_mean) / Hi_training_std
  return y

In [14]:
preprocess_feature(X_train)
preprocess_feature(X_val)
preprocess_feature(X_test)

array([[[-0.12287315,  0.33128065,  0.28214432, ..., -0.34063368,
         -0.99640891,  1.44173226]],

       [[-0.14490139,  0.36419601,  0.25488811, ..., -0.34063368,
         -0.99640891,  1.44173226]],

       [[-0.14490139,  0.43002674,  0.2276319 , ..., -0.42611397,
         -0.99405375,  1.44173226]],

       ...,

       [[-1.42253947,  1.47673532, -1.48950932, ..., -1.62283803,
         -0.92575406,  3.59805475]],

       [[-1.42253947,  1.47673532, -1.51676553, ..., -1.53735774,
         -0.93517471,  3.70472795]],

       [[-1.40051122,  1.47015224, -1.48950932, ..., -1.45187745,
         -0.93517471,  3.63615232]]])

In [15]:
preprocess_output(y_train)
preprocess_output(y_val)
preprocess_output(y_test)

array([[-0.14490139,  0.36419601],
       [-0.14490139,  0.43002674],
       [-0.16692963,  0.49585747],
       ...,
       [-1.42253947,  1.47673532],
       [-1.40051122,  1.47015224],
       [-1.40051122,  1.46356917]])

## One hidden layer **LSTM**

In [16]:
import tensorflow as tf

In [17]:
print("TF version:", tf.__version__)
print("Numpy" + np.__version__)
print("Keras" + tf.keras.__version__)
print("Pandas" + pd.__version__)

TF version: 2.9.2
Numpy1.21.6
Keras2.9.0
Pandas1.3.5


In [18]:
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
#from keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
import os

In [19]:
model = Sequential()
model.add(InputLayer((1, 7)))
model.add(LSTM(8, return_sequences=True))
model.add(LSTM(4, return_sequences=False))
model.add(Dense(2, 'relu'))
model.add(Dense(2, 'linear'))
model.summary()
model.compile(loss='MeanSquaredError', optimizer=Adam(learning_rate=0.003), metrics=[tf.keras.metrics.RootMeanSquaredError()])
history= model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size = 32)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 8)              512       
                                                                 
 lstm_1 (LSTM)               (None, 4)                 208       
                                                                 
 dense (Dense)               (None, 2)                 10        
                                                                 
 dense_1 (Dense)             (None, 2)                 6         
                                                                 
Total params: 736
Trainable params: 736
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
476/476 [==============================] - 12s 8ms/step - loss: 0.4562 - root_mean_squared_error: 0.6754 - val_loss: 0.1881 - val_root_mean_squared_error: 0.4337
Epoch 2/50
476/47

In [21]:
!pip install h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model

In [23]:
model.save('my_model.h5') 

In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()

In [25]:
open("/content/tflite_model.tflite","wb").write(tflite_model) 

19596

In [26]:
!pip install -U tinymlgen

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for tinymlgen: filename=tinymlgen-0.2-py3-none-any.whl size=2242 sha256=2c0fd859f18eae96f387aef70ee499dd73bdd5366fab95a189af2e6c13a542e5
  Stored in directory: /root/.cache/pip/wheels/1c/94/c7/a18049eea10532c9975ad7353fcccc7c436673285b4723a4bb
  Created wheel for hexdump: filename=hexdump-3.3-py3-none-any.whl size=8912 sha256=069dcbe53410fdf267b6af20b3207ce853fd9d327277aaec7c1a5807f8650390
  Stored in directory: /root/.cache/pip/wheels/68/fe/aa/06fd9d6968cfc3e1926eeab1ec728c246dafa120a72ff9afae
Successfully built tinymlgen hexdump


In [27]:
from tinymlgen import port
c_code = port(tflite_model, pretty_print=True)
print(c_code)

AttributeError: ignored

In [28]:
!apt-get -qq install xxd
#!apt-get install -qq xxd #installing the tool

In [ ]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat /content/tflite_model.tflite | xxd -i >> /content/model.h 
!echo "};" >> /content/model.h

#files.download("/content/model.h") #automaticly download your file

In [32]:
#!apt-get  clean && sudo !apt-get  update
#!apt-get -qq install xxd
#!xxd -i arduino_sensor_model.tflite > sensor_model.cc
!xxd -i tflite_model.tflite > sensor_model.h
!cat sensor_model.h

unsigned char tflite_model_tflite[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x14, 0x00, 0x20, 0x00,
  0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00, 0x0c, 0x00, 0x00, 0x00,
  0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0x80, 0x00, 0x00, 0x00, 0xd0, 0x00, 0x00, 0x00, 0x64, 0x13, 0x00, 0x00,
  0x74, 0x13, 0x00, 0x00, 0x80, 0x4a, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x46, 0xc6, 0xff, 0xff,
  0x0c, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00, 0x38, 0x00, 0x00, 0x00,
  0x0f, 0x00, 0x00, 0x00, 0x73, 0x65, 0x72, 0x76, 0x69, 0x6e, 0x67, 0x5f,
  0x64, 0x65, 0x66, 0x61, 0x75, 0x6c, 0x74, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x10, 0xc8, 0xff, 0xff, 0x36, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0x07, 0x00, 0x00, 0x00, 0x64, 0x65, 0x6e, 0x73,
  0x65, 0x5f, 0x31, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x4a, 0xc6, 0xff, 0xff, 0x04, 0x00, 0x00, 0x00, 0x07, 0x00, 0x00, 0x00